<a href="https://colab.research.google.com/github/okana2ki/transcribe/blob/main/transcribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.7 MB/s eta 0:00:00


In [3]:
import os
from docx import Document
from openpyxl import load_workbook

def extract_text_from_table(doc_path):
    doc = Document(doc_path)
    data = {}
    for table in doc.tables:
        for row in table.rows:
            key = row.cells[0].text.strip()
            value = row.cells[1].text.strip().replace('\n', '')
            data[key] = value
    return data

def format_content(format, goals, plan):
    # 教科目計画の内容の各行から番号を取り除き、必要に応じて句読点を挿入
    plan = plan.replace('\n', '、')
    plan = plan[:-1] + '。'  # 最後の改行を句点に変更
    content = f"本科目の授業形態は{format}である。授業の到達目標は次の通りである。{goals}。\n授業計画は次の内容を含む。{plan}"
    return content

def main():
    folder_path = '/content/drive/MyDrive/transcribe'
    excel_path = '/content/drive/MyDrive/transcribe/2_3-1auto.xlsx'
    start_row = 6  # 開始行番号

    # Excel ファイルを読み込む
    wb = load_workbook(excel_path)
    ws = wb.active

    # フォルダ内の全ての Word ファイルを処理
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):
            doc_path = os.path.join(folder_path, filename)
            data = extract_text_from_table(doc_path)
            # フォーマットされた内容を生成
            content = format_content(data['授業形態'], data['授業の到達目標'], data['授業計画・内容'])
            # Excel に書き込み
            ws.cell(row=start_row, column=3).value = data['科目名']
            ws.cell(row=start_row, column=5).value = content
            start_row += 1  # 次の行へ

    # 変更を保存
    wb.save(excel_path)

if __name__ == "__main__":
    main()

KeyError: '授業形態'

In [4]:
import os
from docx import Document
from openpyxl import load_workbook

def extract_text_from_table(doc_path):
    doc = Document(doc_path)
    data = {}
    for table in doc.tables:
        for row in table.rows:
            cells = row.cells
            for i, cell in enumerate(cells):
                text = cell.text.strip()
                if '科目名' in text and i+1 < len(cells):
                    data['科目名'] = cells[i+1].text.strip()
                elif '授業形態' in text and i+1 < len(cells):
                    data['授業形態'] = cells[i+1].text.strip()
                elif '授業の到達目標' in text and i+1 < len(cells):
                    data['授業の到達目標'] = cells[i+1].text.strip().replace('\n', '')
                elif '授業計画・内容' in text and i+1 < len(cells):
                    plan_content = []
                    for j in range(1, 16):  # 1から15回の内容を集約
                        if i+j < len(cells):
                            plan_content.append(cells[i+j].text.strip().replace('\n', '、'))
                    data['授業計画・内容'] = ''.join(plan_content[:-1]) + plan_content[-1].rstrip('、') + '。'
    return data

def format_content(format, goals, plan):
    content = f"本科目の授業形態は{format}である。授業の到達目標は次の通りである。{goals}。\n授業計画は次の内容を含む。{plan}"
    return content

def main():
    folder_path = '/content/drive/MyDrive/transcribe'
    excel_path = '/content/drive/MyDrive/transcribe/2_3-1auto.xlsx'
    start_row = 6  # Excelで書き込みを開始する行番号

    wb = load_workbook(excel_path)
    ws = wb.active

    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):
            doc_path = os.path.join(folder_path, filename)
            data = extract_text_from_table(doc_path)
            if '科目名' in data and '授業形態' in data and '授業の到達目標' in data and '授業計画・内容' in data:
                content = format_content(data['授業形態'], data['授業の到達目標'], data['授業計画・内容'])
                ws.cell(row=start_row, column=3, value=data['科目名'])
                ws.cell(row=start_row, column=5, value=content)
                start_row += 1

    wb.save(excel_path)

if __name__ == "__main__":
    main()